In [52]:
import pandas as pd
import glob
import re
import folium
from folium import plugins
from folium.plugins import HeatMap
import numpy as np
import os

In [53]:
# 矩形範圍的四個角的經度和緯度
#top right(22.62699,120.31145)  bottom left(22.62135,120.29823)

top_right_longitude = 120.31145  # 右上角經度
top_right_latitude = 22.62699  # 右上角緯度
bottom_left_longitude = 120.29823  # 左下角經度
bottom_left_latitude = 22.62135  # 左下角緯度


In [55]:
# import dataframe
chunk_size = 100000
dfs = []
total_df = pd.DataFrame()

file_paths = glob.glob('C:/Users/drink/Desktop/yun/Python/carAccident/2022/*.csv')

for file_path in file_paths:
    reader = pd.read_csv(file_path, chunksize=chunk_size)
    for chunk in reader:
        ###
        # 篩選矩形範圍
        #filtered_df = chunk[
          #  (chunk['經度'] <= top_right_longitude) & (chunk['經度'] >= bottom_left_longitude) &
          #  (chunk['緯度'] >= bottom_left_latitude) & (chunk['緯度'] <= top_right_latitude)
        #]
        
        dfs.append(filtered_df)

total_df = pd.concat(dfs, ignore_index=True)
"""
for file_path in file_paths:
    df = pd.read_csv(file_path)
    total_df = pd.concat([total_df, df], ignore_index=True)


# 去除包含 NaN 或 -1 的行
total_df = total_df.dropna()  # 去除 NaN 的行
total_df = total_df[(total_df != -1).all(axis=1)]  # 去除含有 -1 的行
"""

C:\Users\drink\AppData\Local\Temp\ipykernel_19832\3784638505.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\drink\AppData\Local\Temp\ipykernel_19832\3784638505.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\drink\AppData\Local\Temp\ipykernel_19832\3784638505.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\drink\AppData\Local\Temp\ipykernel_19832\3784638505.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\drink\AppData\Local\Temp\ipykernel_19832\3784638505.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\drink\AppData\Local\Temp\ipykernel_1

'\nfor file_path in file_paths:\n    df = pd.read_csv(file_path)\n    total_df = pd.concat([total_df, df], ignore_index=True)\n\n\n# 去除包含 NaN 或 -1 的行\ntotal_df = total_df.dropna()  # 去除 NaN 的行\ntotal_df = total_df[(total_df != -1).all(axis=1)]  # 去除含有 -1 的行\n'

In [45]:
columns_to_check = ['發生日期','發生時間','處理單位名稱警局層','經度','緯度']

duplicate_records = total_df.duplicated(subset=columns_to_check, keep='first')

# 重複為 True，不重複為 False
total_df['Is_Duplicate'] = duplicate_records

In [46]:
# 確認數據相符度

total_deaths = 0
total_injuries = 0

total_df['死亡受傷人數'] = total_df['死亡受傷人數'].astype(str)

for entry, is_duplicate in zip(total_df['死亡受傷人數'], total_df['Is_Duplicate']):
    if is_duplicate:
        # 使用正則表達式提取數字
        numbers = re.findall(r'\d+', entry)
        
        if numbers:
            # 將提取的數字分别加總到對應的字段
            total_deaths += int(numbers[0])
        if len(numbers) > 1:
            total_injuries += int(numbers[1])

print(f"總死亡人數：{total_deaths}")
print(f"總受傷人數：{total_injuries}")

總死亡人數：0
總受傷人數：568


In [47]:
# 以分號 ';' 拆分 '原始欄位'，並將其分成 '死亡' 和 '受傷' 兩個欄位
total_df[['死亡', '受傷']] = total_df['死亡受傷人數'].str.split(';', expand=True)

In [48]:
"""
majorityTypeCounts = total_df['當事者區分-類別-大類別名稱-車種'].value_counts()

occuredPlaceCounts = total_df['事故位置子類別名稱'].value_counts()

occuredTypeCounts = total_df['事故類型及型態子類別名稱'].value_counts()

causeTpyeCounts = total_df['肇因研判子類別名稱-主要'].value_counts()

ageCounts = total_df['當事者事故發生時年齡'].value_counts()

genderCounts = total_df['當事者屬-性-別名稱'].value_counts()

"""

male_filtered_df = total_df[total_df['當事者屬-性-別名稱'] == '男']

female_filtered_df = total_df[total_df['當事者屬-性-別名稱'] == '女']

maleAgeCounts = male_filtered_df['當事者事故發生時年齡'].value_counts()

femaleAgeCounts = female_filtered_df['當事者事故發生時年齡'].value_counts()


In [50]:

file_path = 'C:/Users/drink/Desktop/yun/Python/carAccident/output.csv'

df_output = pd.concat([majorityTypeCounts, occuredPlaceCounts, occuredTypeCounts, causeTpyeCounts, ageCounts, genderCounts, maleAgeCounts, femaleAgeCounts], axis=1)

df_output.to_csv(file_path, index = True, encoding='big5')


In [ ]:
filtered_df.to_csv('C:/Users/drink/Desktop/yun/Python/carAccident/2022/output2.csv',index = True, encoding='utf-8')

In [ ]:
m0 = folium.Map([23.5,121], zoom_start = 8.5)

In [ ]:

latitudes = total_df['緯度'].to_numpy()
longitudes = total_df['經度'].to_numpy()
weights = total_df['當事者順位'].to_numpy()

# 確保數據都是二維列表
data = np.column_stack((latitudes, longitudes, weights)).tolist()

# 創建熱度圖
HeatMap(data, opacity=0.3).add_to(m0)



In [ ]:
m0

In [ ]:
m0.save("C:/Users/drink/Desktop/yun/Python/carAccident/2022/map11.html")